# Prepare

In [1]:
# machine learning modules
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelBinarizer

import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [2]:
# general modules
import pandas as pd
import sqlite3 
import os
import pandas_profiling
import seaborn as sns
import sqlite3 as sql
import hashlib

In [3]:
# variables
INPUT_FOLDER = "input"
OUTPUT_FOLDER = "output"
DATA_FOLDER = "data"

db_name = "superstore.db"

randon_state = 42

# Import

In [4]:
conn = sql.connect(os.path.join("..", DATA_FOLDER, db_name))

In [5]:
stmt = "select * from superstore"

In [6]:
raw_df = pd.read_sql(stmt, conn, index_col='index')

# Train-Test-Split

In [7]:
df_train, df_test = train_test_split(raw_df, random_state = randon_state)

In [8]:
print(df_train.shape)
print(df_test.shape)

(38471, 23)
(12824, 23)


# Inspect

In [9]:
df_train.head()

,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,...,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,Returned
index,,,,,,,,,,,,,,,,,,,,,
27507,2012-12-27 00:00:00,2012-12-31 00:00:00,Standard Class,SP-20860,a7d03c30d416fc5f7d695b495884fdd7,Corporate,Murfreesboro,Tennessee,United States,37130.0,...,Technology,Accessories,SanDisk Ultra 16 GB MicroSDHC Class 10 Memory ...,72.744,7,0.2,-12.7302,6.720,Medium,No
35511,2014-12-25 00:00:00,2015-01-01 00:00:00,Standard Class,JD-15895,1b2850c124acd1bc24237b4b5228b65e,Corporate,Oosterhout,North Brabant,Netherlands,NaN,...,Office Supplies,Labels,"Smead File Folder Labels, Adjustable",23.730,7,0.5,-21.0000,3.430,Low,No
9172,2012-05-08 00:00:00,2012-05-11 00:00:00,Second Class,AB-10600,6acab08bb2b385c8569adfd24730ee01,Corporate,Phnom Penh,Phnom Penh,Cambodia,NaN,...,Furniture,Bookcases,"Dania Corner Shelving, Pine",617.100,5,0.0,172.6500,36.380,Medium,No
31366,2011-06-30 00:00:00,2011-07-02 00:00:00,Second Class,GH-14410,1528a0a296f3ecf500753855ea9a21a5,Home Office,Lima,Lima (city),Peru,NaN,...,Furniture,Chairs,"Hon Bag Chairs, Red",54.180,3,0.4,-32.5200,4.919,Medium,Yes
24465,2013-06-23 00:00:00,2013-06-26 00:00:00,First Class,KW-16435,648a7c6f93ee0f453ee1378466a84ff8,Consumer,London,England,United Kingdom,NaN,...,Office Supplies,Storage,"Fellowes Box, Wire Frame",50.625,3,0.1,20.2050,8.570,Medium,No


In [10]:
df_train.columns

Index(['Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name',
       'Segment', 'City', 'State', 'Country', 'Postal Code', 'Market',
       'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name',
       'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping Cost',
       'Order Priority', 'Returned'],
      dtype='object')

### Target Column

Time between order and shipment in days

In [11]:
df_train['ship_delay'] = df_train.apply(lambda x: (x["Ship Date"]-x['Order Date']).days, axis=1)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
df_train[["Order Date", "Ship Date", "ship_delay"]].head()

In [ ]:
df_train["ship_delay"].value_counts().plot.bar()

### Profiling

In [ ]:
profile = df_train.profile_report(title='Pandas Profiling Report')

In [ ]:
os.makedirs(os.path.join('..',OUTPUT_FOLDER), exist_ok=True)
profile.to_file(output_file=os.path.join("..", OUTPUT_FOLDER, "data_profile.html"))

### Missings

In [ ]:
df_train.isnull().sum()

Postal Code ==> useless

### Columnwise Inspection

#### Ship Date, Order Date

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [15, 4]
f, axes = plt.subplots(1, 2)

sns.histplot(data=df_train, y="Ship Date", ax=axes[0])
sns.histplot(data=df_train, y="Order Date", ax=axes[1])

#### Ship Mode

In [ ]:
df_train["Ship Mode"].value_counts()

* Predestined for OneHotEncoder

#### Customer ID, Customer Name

In [ ]:
print("Unique values:", len(set(df_train["Customer ID"])))
print("Total values:", len(df_train["Customer ID"]))

In [ ]:
print("Unique values:", len(set(df_train["Customer Name"])))
print("Total values:", len(df_train["Customer Name"]))

* not useless at all... classification?

* customer may have nothing to do with the shipment delay (cannot be caused by the customer)

#### Segment

In [ ]:
df_train["Segment"].value_counts()

=> OneHotEncoder

#### City, State, Country

In [ ]:
print("Unique values City:", len(set(df_train["City"])))
print("Unique values State:", len(set(df_train["State"])))
print("Total values:", len(df_train["State"]))

In [ ]:
df_train["Country"].value_counts()

=> Too high cardinality

#### Postal Code

useless due to missing values

#### Market

In [ ]:
df_train["Market"].value_counts().plot.pie()

* looks interesting, OneHotEncoder; there may be differences in the regions

#### Region

In [ ]:
df_train["Region"].value_counts().plot.pie()

* almost the same as market, but more detailed

#### Product ID, Product Name

In [ ]:
df_train["Product ID"]

In [ ]:
df_train["Product Name"]

* Obviously too detailed

#### Category, Sub-Category

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [21, 5]

f, axes = plt.subplots(1, 2)

sns.countplot(data=df_train, y="Category", ax=axes[0])
sns.countplot(data=df_train, y="Sub-Category", ax=axes[1])

axes[0].tick_params(axis='y', rotation=75, labelsize=12)

* OneHotEncoder

#### Sales, Discount, Profit

In [ ]:
sns.histplot(data=df_train, x="Sales")

In [ ]:
sns.histplot(data=df_train, x="Sales", log_scale=True)

* Log-normal distribution ?

In [ ]:
df_train["Profit"].min()

In [ ]:
df_train["Profit"].max()

In [ ]:
sns.histplot(df_train["Profit"]).set(yscale="log")

#### Quantity

In [ ]:
df_train["Quantity"].value_counts()

#### Discount

In [ ]:
df_train["Discount"].value_counts().plot.bar()

* Percentage, already normalized
* Higher Discount = lower order priority?

#### Shipping Cost

In [ ]:
sns.histplot(data=df_train, x="Shipping Cost", log_scale=True)

#### Order Priority

In [ ]:
df_train["Order Priority"].value_counts()

* OneHotEncoder or OrdinalEncoder
* Order Prio may have impact on shipment delay

#### Returned

In [ ]:
df_train["Returned"].value_counts()

* 0/1, LabelBinarizer
* questionable if it makes sense

### Potential relationships

In [ ]:
plt.rcParams["figure.figsize"] = [10,8]
sns.heatmap(df_train.corr(), vmin=-1, vmax=1, cmap="RdBu", annot=True)

#### Impact of order priority on ship_delay

In [ ]:
pd.crosstab(df_train["ship_delay"], df_train["Order Priority"])

In [ ]:
sns.boxplot(x="Order Priority", y="ship_delay", data=df_train)

* Order Priority has a signifcant impact on the shipment delay

#### Impact of Ship Mode on ship_delay

In [ ]:
sns.boxplot(x="Ship Mode", y="ship_delay", data=df_train)

#### Impact of quantity / sales on ship_delay

In [ ]:
qty_delay = pd.crosstab(df_train["Quantity"], df_train["ship_delay"], normalize='index', margins=True)

In [ ]:
sns.heatmap(qty_delay, cmap="OrRd", annot=True)

In [ ]:
#qty_delay

* insignificant

In [ ]:
bin_labels = [1,2,3,4,5]
df_train['sales_quantiles'] = pd.qcut(df_train['Sales'],
                              q=5,
                              labels=bin_labels)

In [ ]:
sls_delay = pd.crosstab(df_train["sales_quantiles"], df_train["ship_delay"], normalize='index', margins=True)

In [ ]:
sns.heatmap(sls_delay, cmap="OrRd", annot=True)

* insignificant

In [ ]:
# -----------------------------------------------------------------

In [ ]:
# select potentially useful columns
#useful_columns = [""]
#df_inspect = df_train[useful_columns]

In [ ]:
#df_inspect.reset_index(drop=True, inplace=True)


In [ ]:
#df_inspect.loc[:,""] = LabelBinarizer().fit_transform(df_inspect["..."])

In [ ]:
#sns.pairplot(data=df_inspect)

In [ ]:
#sns.heatmap(df_inspect.corr())

In [ ]:
#model = smf.ols(formula="target ~ a + b + c", data=df_inspect)
#results = model.fit()
#print(results.summary())